In [7]:
# Install dependencies (run once per environment)
%pip install -q dspy python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [14]:
# Basic imports and environment setup
import os
import dspy
from dotenv import load_dotenv
# from dspy.adapters import JSONAdapter
from dspy import History

# Load API keys from .env (ANTHROPIC_API_KEY is expected, already set in your env)
load_dotenv()

# Configure model provider to match LangGraph (Claude 3.5 Sonnet)
lm = dspy.LM("anthropic/claude-3-5-sonnet-20241022", api_key=os.getenv("ANTHROPIC_API_KEY"), temperature=0.1, max_tokens=4000)
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)

# Configure DSPy with LM only (ReAct handles its own formatting)
dspy.configure(lm=lm)
# dspy.configure(adapter=JSONAdapter())

# Conversation memory
conversation_history = History(messages=[])

print("DSPy configured with Claude. Ready to build customer support agent.")


DSPy configured with Claude. Ready to build customer support agent.


In [15]:
# Tools: Mock orders DB and documentation helpers
from pathlib import Path
from typing import Optional, Literal
from dataclasses import dataclass

# Documentation category literal
DocumentationCategory = Literal["shipping", "returns", "products", "account", "payment"]

# Mock orders database
ORDERS_DATABASE = [
    {
        "order_id": "ORD-001",
        "customer_email": "john.doe@email.com",
        "customer_name": "John Doe",
        "product": "Wireless Headphones",
        "price": 99.99,
        "status": "delivered",
        "order_date": "2024-01-15",
        "delivery_date": "2024-01-18",
    },
    {
        "order_id": "ORD-002",
        "customer_email": "jane.smith@email.com",
        "customer_name": "Jane Smith",
        "product": "Smart Watch",
        "price": 199.99,
        "status": "shipped",
        "order_date": "2024-01-20",
        "delivery_date": None,
    },
    {
        "order_id": "ORD-003",
        "customer_email": "bob.wilson@email.com",
        "customer_name": "Bob Wilson",
        "product": "Laptop Stand",
        "price": 49.99,
        "status": "processing",
        "order_date": "2024-01-22",
        "delivery_date": None,
    },
    {
        "order_id": "ORD-004",
        "customer_email": "alice.brown@email.com",
        "customer_name": "Alice Brown",
        "product": "Bluetooth Speaker",
        "price": 79.99,
        "status": "delivered",
        "order_date": "2024-01-10",
        "delivery_date": "2024-01-13",
    },
    {
        "order_id": "ORD-005",
        "customer_email": "charlie.davis@email.com",
        "customer_name": "Charlie Davis",
        "product": "Phone Case",
        "price": 24.99,
        "status": "cancelled",
        "order_date": "2024-01-25",
        "delivery_date": None,
    },
]

# Documentation directory (local to this notebook)
try:
    BASE_DIR = Path(__file__).parent  # type: ignore[name-defined]
except NameError:
    BASE_DIR = Path().resolve()
DOCS_DIR = BASE_DIR / "documentation"


def _load_documentation_file(category: DocumentationCategory) -> str:
    """Load documentation content from the repo .txt files."""
    file_path = DOCS_DIR / f"{category}.txt"
    try:
        if file_path.exists():
            return file_path.read_text(encoding="utf-8").strip()
        return f"No documentation file found for category: {category}"
    except Exception as e:
        return f"Error reading documentation file for {category}: {e}"


def _classify_query_to_category(query: str) -> DocumentationCategory:
    """Heuristic classification."""
    q = query.lower()
    if any(k in q for k in ["shipping", "delivery", "express", "standard", "overnight", "tracking"]):
        return "shipping"
    if any(k in q for k in ["return", "refund", "exchange", "policy", "rma"]):
        return "returns"
    if any(k in q for k in ["product", "warranty", "compatibility", "specification", "specs", "feature"]):
        return "products"
    if any(k in q for k in ["account", "password", "login", "profile", "dashboard", "reset"]):
        return "account"
    if any(k in q for k in ["payment", "billing", "credit", "card", "paypal", "apple pay", "invoice", "receipt"]):
        return "payment"
    return "products"


def is_relevant_query(query: str) -> bool:
    """Check if the query is ecommerce-related."""
    q = query.lower()
    keywords = [
        "order", "purchase", "buy", "bought", "ordered", "order id", "order number",
        "tracking", "delivery", "shipped", "delivered", "status",
        "product", "item", "specification", "compatibility", "warranty",
        "return", "refund", "exchange", "defective", "broken", "damaged",
        "account", "login", "password", "profile", "billing", "payment",
        "credit card", "paypal", "invoice", "receipt",
        "shipping", "express", "standard", "overnight", "cost", "free shipping",
        "address", "package", "shipment", "help", "support", "customer service",
        "assistance", "problem", "issue", "question", "inquiry", "complaint", "concern",
    ]
    return any(k in q for k in keywords)


In [16]:
# Tools: plain functions (simple, no wrappers)

def doc_search(query: str, category: str = "auto") -> str:
    """Return relevant documentation text for a category, inferred from the query if needed."""
    resolved = _classify_query_to_category(query) if category == "auto" else category  # type: ignore[assignment]
    content = _load_documentation_file(resolved)  # type: ignore[arg-type]
    if not content or "No documentation file found" in content or "Error reading" in content:
        return f"I couldn't find documentation for '{resolved}'."

    MAX_CONTENT_LENGTH = 8000
    if len(content) > MAX_CONTENT_LENGTH:
        trimmed = content[:MAX_CONTENT_LENGTH]
        last = max(trimmed.rfind("."), trimmed.rfind("\n"))
        if last > 0:
            trimmed = content[:last]
        content = trimmed

    header = f"Documentation ({resolved})\n\n"
    trailer = f"\n\n---\nUSER QUERY: {query}"
    return header + content + trailer


def search_orders(customer_email: str = "", order_id: str = "") -> str:
    """Find orders by email or order ID in the mock database."""
    if not customer_email and not order_id:
        return "Please provide either a customer email or order ID to search for orders."

    found = []
    for o in ORDERS_DATABASE:
        if customer_email and o["customer_email"].lower() == customer_email.lower():
            found.append(o)
        elif order_id and o["order_id"].upper() == order_id.upper():
            found.append(o)

    if not found:
        if customer_email:
            return f"No orders found for email: {customer_email}"
        return f"No orders found for order ID: {order_id}"

    chunks = []
    for o in found:
        chunks.append(
            (
                f"Order ID: {o['order_id']}\n"
                f"Customer: {o['customer_name']} ({o['customer_email']})\n"
                f"Product: {o['product']}\n"
                f"Price: ${o['price']}\n"
                f"Status: {o['status'].title()}\n"
                f"Order Date: {o['order_date']}\n"
                f"Delivery Date: {o['delivery_date'] if o['delivery_date'] else 'Not delivered yet'}\n"
            ).strip()
        )
    return "\n\n".join(chunks)


def refund(order_id: str, reason: str = "Customer request") -> str:
    """Process a refund for the given order if eligible."""
    order = None
    for o in ORDERS_DATABASE:
        if o["order_id"].upper() == order_id.upper():
            order = o
            break
    if not order:
        return f"Order {order_id} not found. Please verify the order ID."
    if order["status"] == "cancelled":
        return f"Order {order_id} has already been cancelled and cannot be refunded."
    if order["status"] == "processing":
        return (
            f"Order {order_id} is still being processed. Refunds can only be processed "
            "for shipped or delivered orders."
        )

    order["status"] = "refunded"
    refund_amount = order["price"]
    return (
        "Refund processed successfully!\n\n"
        f"Order ID: {order_id}\n"
        f"Customer: {order['customer_name']} ({order['customer_email']})\n"
        f"Product: {order['product']}\n"
        f"Refund Amount: ${refund_amount}\n"
        f"Reason: {reason}\n"
        "Status: Refunded\n\n"
        "The refund will be credited to the original payment method within 5-7 business days."
    )


def get_status(order_id: str) -> str:
    """Return a simple status summary for the given order ID."""
    for o in ORDERS_DATABASE:
        if o["order_id"].upper() == order_id.upper():
            details = (
                f"Order ID: {o['order_id']}\n"
                f"Status: {o['status'].title()}\n"
                f"Customer: {o['customer_name']}\n"
                f"Product: {o['product']}\n"
                f"Order Date: {o['order_date']}\n"
            )
            details += (
                f"Delivery Date: {o['delivery_date']}" if o["delivery_date"] else "Delivery Date: Not delivered yet"
            )
            return details.strip()
    return f"Order {order_id} not found. Please verify the order ID."

In [17]:
# Utilities: sanitize History to avoid empty Anthropic content blocks
from dspy import History as _History

def sanitize_history(history: _History) -> _History:
    """Return a new History with only valid, non-empty messages.
    - Converts legacy entries with keys 'user_message'/'answer' into role/content
    - Drops messages with empty/whitespace-only content
    """
    new_messages = []
    removed_count = 0
    converted_count = 0

    for m in getattr(history, "messages", []):
        if not isinstance(m, dict):
            removed_count += 1
            continue
        role = m.get("role")
        content = m.get("content")

        if role in ("user", "assistant") and isinstance(content, str) and content.strip():
            new_messages.append({"role": role, "content": content})
            continue

        # Convert legacy shapes from earlier notebook versions
        if isinstance(m.get("user_message"), str) and m["user_message"].strip():
            new_messages.append({"role": "user", "content": m["user_message"]})
            converted_count += 1
            continue
        if isinstance(m.get("answer"), str) and m["answer"].strip():
            new_messages.append({"role": "assistant", "content": m["answer"]})
            converted_count += 1
            continue

        removed_count += 1

    if removed_count or converted_count:
        print(f"[debug] sanitize_history: kept={len(new_messages)} converted={converted_count} removed={removed_count}")
        for idx, last in enumerate(new_messages[-2:], start=1):
            print(f"[debug] sanitize_history:last{idx} role={last['role']} len={len(last['content'])}")

    return _History(messages=new_messages)


In [18]:
# ReAct agent with tools + History + JSON output

# Provide tools as simple functions to ReAct
# See definitions above: doc_search, search_orders, refund, get_status

class SupportReActSignature(dspy.Signature):
    """
    You are a professional customer service representative for TechStore, an e-commerce company specializing in electronics and tech accessories.
    Use provided customer context to personalize responses.
    Guidelines: Verify order IDs before refunds; use documentation for policy info; search orders for lookups; use status for quick checks; escalate politely when needed.

    You can call tools: doc_search, search_orders, refund, get_status.
    When finished, produce:
    - `action`: the primary tool used (one of: doc_search, search_orders, refund, get_status, answer_direct)
    - `tool_result`: the most relevant tool output you used (may be empty for answer_direct)
    - `answer`: the final customer-facing reply
    Keep responses concise and professional.
    """
    user_message: str = dspy.InputField(description="The customer's message")
    customer_email: str = dspy.InputField(description="Customer email if available")
    order_id: str = dspy.InputField(description="Order ID if available")
    history: dspy.History = dspy.InputField(description="Conversation history")

    reasoning: str = dspy.OutputField(description="Brief plan and justification")
    action: str = dspy.OutputField(description="Chosen action/tool")
    tool_result: str = dspy.OutputField(description="Tool output used to answer")
    answer: str = dspy.OutputField(description="Final answer")


react_agent = dspy.ReAct(
    SupportReActSignature,
    tools=[doc_search, search_orders, refund, get_status],
    max_iters=3,
)


def run_support_agent(user_message: str, customer_email: str = "", order_id: str = "") -> dict:
    """Single entrypoint that runs the ReAct agent and returns a JSON-serializable dict."""
    # Guard against empty/whitespace user input (avoid empty content blocks)
    if not isinstance(user_message, str) or not user_message.strip():
        return {"answer": "", "action": "reject", "tool_result": ""}

    if user_message and not is_relevant_query(user_message):
        rejection = (
            "I'm sorry, but I can only assist with e-commerce related inquiries such as order status, "
            "product information, shipping, returns, refunds, account and payment issues."
        )
        conversation_history.messages.append({"role": "user", "content": user_message})
        conversation_history.messages.append({"role": "assistant", "content": rejection})
        return {"answer": rejection, "action": "reject", "tool_result": ""}

    # Maintain history and sanitize before calling model
    conversation_history.messages.append({"role": "user", "content": user_message})
    safe_history = sanitize_history(conversation_history)

    result = react_agent(
        user_message=user_message,
        customer_email=customer_email,
        order_id=order_id,
        history=safe_history,
    )

    answer = getattr(result, "answer", "")
    action = getattr(result, "action", "")
    tool_result = getattr(result, "tool_result", "")

    # Only append non-empty assistant messages
    if isinstance(answer, str) and answer.strip():
        conversation_history.messages.append({"role": "assistant", "content": answer})

    return {"answer": answer, "action": action, "tool_result": tool_result}

print("ReAct support agent ready.")


ReAct support agent ready.


In [19]:
# Example: minimal required inputs using ReAct agent returning JSON

input_payload = {
    "messages": [
        {"role": "human", "content": "What's the status of order ORD-001?"}
    ],
    "customer_email": "john.doe@email.com",
    "order_id": "ORD-001",
}

user_text = input_payload["messages"][0]["content"]
resp = run_support_agent(user_message=user_text, customer_email=input_payload["customer_email"], order_id=input_payload["order_id"])
print(resp)


{'answer': 'Hi John — I checked order ORD-001 (Wireless Headphones). It shows as Delivered on 2024-01-18.\n\nIf you received the item and everything’s OK, no further action is needed. If you haven’t received it, it’s damaged, or you’d like a return/replacement, reply here and I’ll open an investigation or start the return for you.\n\n— TechStore Support', 'action': 'get_status', 'tool_result': 'Order ID: ORD-001\nStatus: Delivered\nCustomer: John Doe\nProduct: Wireless Headphones\nOrder Date: 2024-01-15\nDelivery Date: 2024-01-18'}


In [20]:
# Inspect the underlying prompts used by DSPy components
lm.inspect_history(n=3)






[2025-09-17T15:17:54.941253]

System message:

Your input fields are:
1. `user_message` (str): The customer's message
2. `customer_email` (str): Customer email if available
3. `order_id` (str): Order ID if available
4. `history` (History): Conversation history
5. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['doc_search', 'search_orders', 'refund', 'get_status', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_message ## ]]
{user_message}

[[ ## customer_email ## ]]
{customer_email}

[[ ## order_id ## ]]
{order_id}

[[ ## history ## ]]
{history}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: doc_search; search_orders; refund; get_status; finish

[[ ## next_tool_

In [21]:
# Additional examples using ReAct + JSON

# 1) Documentation question
print("\n--- Doc question ---")
msg = "What's your return policy?"
print(run_support_agent(user_message=msg, customer_email="", order_id=""))

# 2) Orders lookup by email
print("\n--- Search orders by email ---")
print(run_support_agent(user_message="Can you find my orders?", customer_email="jane.smith@email.com"))

# 3) Refund path (eligible)
print("\n--- Refund ---")
print(run_support_agent(user_message="I want a refund", customer_email="john.doe@email.com", order_id="ORD-001"))

# 4) Irrelevant query
print("\n--- Irrelevant ---")
print(run_support_agent(user_message="Tell me a joke about quantum cats"))



--- Doc question ---
{'answer': 'Here’s a quick summary of our return policy:\n\n- Window: 30 days from delivery for returns or exchanges.\n- Condition: Items must be unused and in their original packaging.\n- How: Contact customer service with your order number to get a return authorization (RA) number and return label, pack the item with the RA visible, and ship to our returns center.\n- Refunds: Processed within 5–7 business days after we receive the return; credited to the original payment method.\n- Exchanges: Available (same 30-day window), subject to availability.\n- Shipping: We provide prepaid return labels for most items. If you use your own shipping, you’re responsible for costs unless the item is damaged or defective.\n- Non-returnable: Personalized/custom items, items without original packaging, used/damaged items, gift cards, digital downloads, and perishables.\n- Damaged/defective: Contact us immediately — we’ll arrange a replacement or full refund at no cost.\n\nIf you